In [1]:
import pandas as pd
import numpy as np
from numpy.random import randint, shuffle
import random

def get_multiple_dfs(data_frame_rows):
    """
    Get a complete code str that creates a DF with random value
    """
    sizes_before_join = int(data_frame_rows * 1.1)
    start_with_offset = int(data_frame_rows * 0.1)
    end_with_offset = start_with_offset + sizes_before_join
    assert sizes_before_join - start_with_offset == data_frame_rows

    id_a = np.arange(sizes_before_join)
    shuffle(id_a)
    a = randint(0,100,size=(sizes_before_join))
    b = randint(0,100,size=(sizes_before_join))
    categories = ['cat_a', 'cat_b', 'cat_c']
    group_col_1 = pd.Series(random.choices(categories, k=sizes_before_join))
    group_col_2 = pd.Series(random.choices(categories, k=sizes_before_join))
    group_col_3 = pd.Series(random.choices(categories, k=sizes_before_join))

    id_b = np.arange(start_with_offset, end_with_offset)
    shuffle(id_b)
    c = randint(0,100,size=(sizes_before_join)) 
    d = randint(0,100,size=(sizes_before_join))

    df_a = pd.DataFrame(zip(id_a, a, b, group_col_1, group_col_2, group_col_3), columns=['id', 'A', 'B', 
        'group_col_1', 'group_col_2', 'group_col_3'])
    df_b = pd.DataFrame(zip(id_b, c, d), columns=['id', 'C', 'D'])
        
    return df_a, df_b

In [2]:
df_a, df_b = get_multiple_dfs(1000000)

# Without instrumentation

In [3]:
%%time
test = df_a.merge(df_b, on='id')

CPU times: user 322 ms, sys: 70.6 ms, total: 392 ms
Wall time: 392 ms


In [4]:
from mlinspect.inspections._lineage import LineageId, JoinLineageId

lineage_id_list_a = [LineageId(0, row_id) for row_id in range(len(df_a))]
lineage_ids_a = pd.DataFrame({"RowLineage(10)": pd.Series(lineage_id_list_a, dtype="object")})
lineage_id_list_b = [LineageId(0, row_id) for row_id in range(len(df_b))]
lineage_ids_b = pd.DataFrame({"RowLineage(10)": pd.Series(lineage_id_list_b, dtype="object")})

In [5]:
from mlinspect.backends._pandas_backend import PandasBackend, execute_inspection_visits_join
from mlinspect.backends._pandas_backend_frame_wrapper import MlinspectDataFrame
from mlinspect.inspections import RowLineage
from mlinspect.inspections._inspection_input import OperatorContext
from mlinspect.instrumentation._dag_node import OperatorType, CodeReference

pandas_backend = PandasBackend()
pandas_backend.inspections = [RowLineage(10)]
function_info = ('pandas.core.frame', 'merge')
operator_context = OperatorContext(OperatorType.JOIN, function_info)
code_reference = CodeReference(0,0,0,10)

mlinspect_df_a = MlinspectDataFrame(df_a.copy())
mlinspect_df_a.annotations = lineage_ids_a
mlinspect_df_b = MlinspectDataFrame(df_b.copy())
mlinspect_df_b.annotations = lineage_ids_b

# With current instrumentation

In [6]:
%%time
mlinspect_df_a['mlinspect_index_x'] = range(0, len(mlinspect_df_a))
mlinspect_df_b['mlinspect_index_y'] = range(0, len(mlinspect_df_b))
original_return_value = mlinspect_df_a.merge(mlinspect_df_b, on='id')


return_value = execute_inspection_visits_join(pandas_backend, operator_context, code_reference,
                                              mlinspect_df_a,
                                              mlinspect_df_a.annotations,
                                              mlinspect_df_b,
                                              mlinspect_df_b.annotations,
                                              original_return_value)

# list(pandas_backend.dag_node_identifier_to_inspection_output.values())
# return_value

CPU times: user 8.99 s, sys: 887 ms, total: 9.88 s
Wall time: 9.92 s


# Using apply, not a fast function

In [13]:
%%time
df_a['lineage_left'] = lineage_ids_a
df_b['lineage_right'] = lineage_ids_b

join_result = df_a.merge(df_b, on='id')

join_result['annotations'] = join_result.apply(lambda row: JoinLineageId([row.lineage_left, row.lineage_right]), axis=1)
join_result.drop('lineage_left', inplace=True, axis=1)
join_result.drop('lineage_right', inplace=True, axis=1)
dag_annotation = join_result.head(10)
annotation = join_result.pop('annotations')

CPU times: user 22.8 s, sys: 425 ms, total: 23.2 s
Wall time: 23.3 s


In [14]:
dag_annotation

,id,A,B,group_col_1,group_col_2,group_col_3,C,D,annotations
0,505546,75,62,cat_a,cat_a,cat_a,3,22,JoinLineageId(lineage_ids=[LineageId(operator_...
1,1062648,15,50,cat_a,cat_a,cat_c,13,58,JoinLineageId(lineage_ids=[LineageId(operator_...
2,1035890,65,10,cat_c,cat_c,cat_c,86,0,JoinLineageId(lineage_ids=[LineageId(operator_...
3,261211,98,32,cat_c,cat_c,cat_a,26,36,JoinLineageId(lineage_ids=[LineageId(operator_...
4,762999,61,88,cat_c,cat_b,cat_c,18,37,JoinLineageId(lineage_ids=[LineageId(operator_...
5,662830,45,23,cat_c,cat_a,cat_c,23,58,JoinLineageId(lineage_ids=[LineageId(operator_...
6,496724,75,91,cat_c,cat_c,cat_a,14,29,JoinLineageId(lineage_ids=[LineageId(operator_...
7,192244,59,94,cat_c,cat_c,cat_b,49,1,JoinLineageId(lineage_ids=[LineageId(operator_...
8,147918,91,34,cat_a,cat_b,cat_b,9,90,JoinLineageId(lineage_ids=[LineageId(operator_...
9,545428,47,74,cat_a,cat_a,cat_b,73,80,JoinLineageId(lineage_ids=[LineageId(operator_...
